In [2]:
import numpy as np
import math as m
from scipy.spatial.distance import cdist
from sklearn.datasets import make_blobs
import scipy as sp
from numba import njit

In [75]:
@njit
def hand_norm(A):
    return m.sqrt(np.sum(A ** 2))

@njit
def hand_scalar_prod(A,B):
    prod = np.zeros((len(A)))
    k = 0
    for a,b in (zip(A,B)):
        prod[k]= a * b 
        k +=1
        
    return np.sum(prod)

@njit
def hand_dist(A,B, metric = 'euclidean'):
    dist = np.zeros((len(A),(len(A))))
    if metric == 'euclidean':
        for i in range(len(A)):
            for ii in range(len(B)):
                dist[ii,i] = m.sqrt(np.sum((A[i,:] - B[ii,:]) ** 2))

    if metric == 'cosine':
        for i in range(len(A)):
            for ii in range(len(B)):
                dist[ii,i] = 1 - (hand_scalar_prod(A[i,:],B[ii,:])/(hand_norm(A[i,:])*hand_norm(B[ii,:])))
            
    if metric == 'mahalanobis':
        concat = np.zeros((len(A)+len(B),len(A[0])))
        concat[:len(A)] = A
        concat[len(A):] = B        
        VI = np.linalg.inv(np.cov(concat.T)).T
        for i in range(len(A)):
            for ii in range(len(B)):
                dist[ii,i] = np.sqrt(np.dot(np.dot((A[i,:]-B[ii,:]),VI),(A[i,:]-B[ii,:]).T))
            
    return dist

In [76]:
centers = [(-5, -5), (0, 0), (5, 5)]
data, _ = make_blobs(n_samples=10, centers=centers, shuffle=False,
                  random_state=42)

In [77]:
data

array([[-4.50328585, -5.1382643 ],
       [-4.35231146, -3.47697014],
       [-5.23415337, -5.23413696],
       [-3.42078718, -4.23256527],
       [-0.46947439,  0.54256004],
       [-0.46341769, -0.46572975],
       [ 0.24196227, -1.91328024],
       [ 3.27508217,  4.43771247],
       [ 3.98716888,  5.31424733],
       [ 4.09197592,  3.5876963 ]])

In [78]:
scipy = cdist(data, data, metric='euclidean')

In [80]:
thiago = hand_dist(data, data, metric='euclidean')

In [81]:
thiago - scipy

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [84]:
scipy = cdist(data, data, metric='cosine')

In [85]:
thiago = hand_dist(data, data, metric='cosine')

In [86]:
thiago - scipy

array([[-2.22044605e-16,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        -2.22044605e-16,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.

In [87]:
scipy = cdist(data, data, metric='mahalanobis')

In [88]:
thiago = hand_dist(data, data, metric='mahalanobis')

In [90]:
thiago - scipy

array([[ 0.00000000e+00,  3.55271368e-15,  1.55431223e-15,
         3.33066907e-16,  2.44249065e-15, -1.33226763e-15,
         2.66453526e-15, -2.66453526e-15, -3.55271368e-15,
        -8.88178420e-16],
       [ 3.55271368e-15,  0.00000000e+00,  1.99840144e-15,
         3.77475828e-15, -1.33226763e-15,  1.33226763e-15,
         6.66133815e-15, -1.33226763e-15,  1.33226763e-15,
         4.44089210e-15],
       [ 1.55431223e-15,  1.99840144e-15,  0.00000000e+00,
         2.22044605e-15,  2.44249065e-15,  1.99840144e-15,
         3.99680289e-15,  4.44089210e-16, -1.77635684e-15,
         4.44089210e-16],
       [ 3.33066907e-16,  3.77475828e-15,  2.22044605e-15,
         0.00000000e+00,  3.99680289e-15,  1.11022302e-15,
         3.33066907e-15,  2.22044605e-15,  8.88178420e-16,
         1.33226763e-15],
       [ 2.44249065e-15, -1.33226763e-15,  2.44249065e-15,
         3.99680289e-15,  0.00000000e+00,  2.44249065e-15,
         7.54951657e-15,  2.22044605e-16, -1.11022302e-15,
         3.